<a href="https://colab.research.google.com/github/Morrissenzeyi/KNN-and-Naive-bayes-IP/blob/main/scripting_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# importing libraries
import pandas as pd
pd.options.display.float_format = "{:.2f}".format
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as lda
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures
import statsmodels.regression.quantile_regression as quant
import scipy as sp
import warnings
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
test = pd.read_csv("/content/test (1).csv")
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.50,0,0,330911,7.83,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.00,1,0,363272,7.00,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.00,0,0,240276,9.69,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.00,0,0,315154,8.66,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.00,1,1,3101298,12.29,NaN,S


In [3]:
train = pd.read_csv("/content/train (5).csv")
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.00,1,0,A/5 21171,7.25,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.00,1,0,PC 17599,71.28,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.00,0,0,STON/O2. 3101282,7.92,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.00,1,0,113803,53.10,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.00,0,0,373450,8.05,NaN,S


In [4]:
# replacing age NaN WITH MEAN values of age
train['Age'].fillna(train['Age'].mean(),inplace=True)
# drop the cabin column
train = train.drop(columns=['Cabin'])
train = train.dropna()

In [5]:
# Identifying the duplicate Data

len(train)-len(train.drop_duplicates())

0

In [6]:
# Checking for Outliers
train1 = train[['Survived', 'Pclass', 'Age', 'SibSp',
       'Parch', 'Fare']]
Q1 = train1.quantile(0.25)
Q3 = train1.quantile(0.75)
IQR = Q3 - Q1
((train1 < (Q1 - 1.5 * IQR)) | (train1 > (Q3 + 1.5 * IQR))).sum()
# we can't drop all this outliers it can affect our work by reducing the quality of the dataset.

Survived      0
Pclass        0
Age          65
SibSp        46
Parch       213
Fare        114
dtype: int64

In [7]:
# Encoding
cat_cols = ['Sex']

for col in cat_cols:
  cat_enc = {'Sex':{'male':0,'female':1}}

train.replace(cat_enc, inplace=True)

train['Sex'].astype(int)

cat_cols = ['Embarked']

for col in cat_cols:
  cat_enc = {'Embarked':{'S':0,'C':1,'Q':2}}

train.replace(cat_enc, inplace=True)

train['Embarked'].astype(int)

0      0
1      1
2      0
3      0
4      0
      ..
886    0
887    0
888    0
889    1
890    2
Name: Embarked, Length: 889, dtype: int64

In [8]:
# checking for missing values
test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [9]:
# replacing age NaN WITH MEAN values of age
test['Age'].fillna(test['Age'].mean(), inplace=True)

In [10]:
# Dropping cabin column
test = test.drop(columns=['Cabin'])

In [11]:
# Identifying the duplicate Data

len(test)-len(test.drop_duplicates())

0

In [12]:
# Checking for Outliers
test1 = test[[ 'Pclass', 'Age', 'SibSp',
       'Parch', 'Fare']]
Q1 = test1.quantile(0.25)
Q3 = test1.quantile(0.75)
IQR = Q3 - Q1
((train1 < (Q1 - 1.5 * IQR)) | (train1 > (Q3 + 1.5 * IQR))).sum()
# we can't drop all this outliers it can affect our work by reducing the quality of the dataset.

Age          71
Fare        112
Parch       213
Pclass        0
SibSp        46
Survived      0
dtype: int64

In [13]:
# encoding the Sex column

cat_cols = ['Sex']

for col in cat_cols:
  cat_enc = {'Sex':{'male':0,'female':1}}

test.replace(cat_enc, inplace=True)

test['Sex'].astype(int)


cat_cols = ['Embarked']

for col in cat_cols:
  cat_enc = {'Embarked':{'S':0,'C':1,'Q':2}}

test.replace(cat_enc, inplace=True)

test['Embarked'].astype(int)

test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,892,3,"Kelly, Mr. James",0,34.50,0,0,330911,7.83,2
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",1,47.00,1,0,363272,7.00,0
2,894,2,"Myles, Mr. Thomas Francis",0,62.00,0,0,240276,9.69,2
3,895,3,"Wirz, Mr. Albert",0,27.00,0,0,315154,8.66,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22.00,1,1,3101298,12.29,0


In [14]:
# Split the independent and dependent variables
X = train[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare','Embarked']]
y = train['Survived']

# Train using 80% of the data.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [15]:
# standardizing the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit_transform(X)

array([[ 0.82520863, -0.73534203, -0.59049493, ..., -0.47432585,
        -0.50023975, -0.56968402],
       [-1.57221121,  1.35991138,  0.64397101, ..., -0.47432585,
         0.78894661,  1.00313924],
       [ 0.82520863,  1.35991138, -0.28187844, ..., -0.47432585,
        -0.48664993, -0.56968402],
       ...,
       [ 0.82520863,  1.35991138,  0.00352373, ...,  2.00611934,
        -0.17408416, -0.56968402],
       [-1.57221121, -0.73534203, -0.28187844, ..., -0.47432585,
        -0.0422126 ,  1.00313924],
       [ 0.82520863, -0.73534203,  0.18104628, ..., -0.47432585,
        -0.49017322,  2.5759625 ]])

In [16]:
# splitting into train an test sets

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state=0)

# setting the dicision tree classifier

# Modelling for training and predictions

clf = KNeighborsClassifier(n_neighbors=3)
clf = clf.fit(X_train, y_train)

# predicting using the model built

y_pred = clf.predict(X_test)

In [17]:
# comparing the actual and predicted

comparison_frame = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

comparison_frame.describe()

,Actual,Predicted
count,178.00,178.00
mean,0.41,0.36
std,0.49,0.48
min,0.00,0.00
25%,0.00,0.00
50%,0.00,0.00
75%,1.00,1.00
max,1.00,1.00


In [18]:
# Evaluating the Algorithm
print('The accuracy of the model is ',metrics.accuracy_score(y_test, y_pred))

The accuracy of the model is  0.6348314606741573


**Grid search model**

In [19]:
#Defining the hyper parameters
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p = [1,2]
metric = ['manhattan', 'euclidean', 'minkowski']
#Creating a dictionary with the hyperparameters
hyperparameters = dict(leaf_size = leaf_size, n_neighbors = n_neighbors, p=p,metric = metric)

In [20]:
# let's use GridSearch to look for the best parameters
from sklearn.model_selection import GridSearchCV

classifier = KNeighborsClassifier()

clf = GridSearchCV(classifier, hyperparameters,cv=10)
clf = clf.fit(X_train, y_train)

print('\n')

print(clf.best_params_)
print('\n')
#Creating a better model using what the parameters has given to us by the gridsearch
modelone = KNeighborsClassifier(leaf_size = clf.best_params_['leaf_size'], n_neighbors = clf.best_params_['n_neighbors'], p = clf.best_params_['p'], metric = clf.best_params_['metric']).fit(X_train, y_train)
modelone

# predicting using the model built

y_pred = modelone.predict(X_test)



{'leaf_size': 40, 'metric': 'manhattan', 'n_neighbors': 3, 'p': 1}




In [21]:
# comparing the actual and predicted

comparison_frame = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print('\n')
print(comparison_frame)
print('\n')
print(comparison_frame.describe())



     Actual  Predicted
14        0          1
159       0          0
763       1          1
741       0          1
483       1          0
..      ...        ...
886       0          0
673       1          0
182       0          0
632       1          0
141       1          1

[178 rows x 2 columns]


       Actual  Predicted
count  178.00     178.00
mean     0.41       0.38
std      0.49       0.49
min      0.00       0.00
25%      0.00       0.00
50%      0.00       0.00
75%      1.00       1.00
max      1.00       1.00


In [22]:
print('The accuracy of the model is ',metrics.accuracy_score(y_test, y_pred))
print('\n', 'Confusion matrix')
print(confusion_matrix(y_test, y_pred))
print('\n', 'Classification report')
print(classification_report(y_test, y_pred))

The accuracy of the model is  0.6629213483146067

 Confusion matrix
[[78 27]
 [33 40]]

 Classification report
              precision    recall  f1-score   support

           0       0.70      0.74      0.72       105
           1       0.60      0.55      0.57        73

    accuracy                           0.66       178
   macro avg       0.65      0.65      0.65       178
weighted avg       0.66      0.66      0.66       178



**Baseline Model**

In [23]:
# split the data into labels and features

X = train[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare','Embarked']]
y = train['Survived']

# standardizing the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit_transform(X)


# splitting into train an test sets

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state=0)

# setting the dicision tree classifier

# Modelling for training and predictions

clf = KNeighborsClassifier(n_neighbors=3)
clf = clf.fit(X_train, y_train)

# predicting using the model built
y_pred = clf.predict(X_test)

# comparing the actual and predicted

comparison_frame = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
comparison_frame.describe()
print('The accuracy of the model is ',metrics.accuracy_score(y_test, y_pred))
print('\n', 'Confusion matrix')
print(confusion_matrix(y_test, y_pred))
print('\n', 'Classification report')
print(classification_report(y_test, y_pred))

The accuracy of the model is  0.6404494382022472

 Confusion matrix
[[116  41]
 [ 55  55]]

 Classification report
              precision    recall  f1-score   support

           0       0.68      0.74      0.71       157
           1       0.57      0.50      0.53       110

    accuracy                           0.64       267
   macro avg       0.63      0.62      0.62       267
weighted avg       0.63      0.64      0.64       267



**KNN with Grid search Model 70/30**

In [24]:
#Defining the hyper parameters for the Knearest Neighbors Classifier
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p = [1,2]
metric = ['manhattan', 'euclidean', 'minkowski']
#Creating a dictionary with the hyperparameters
hyperparameters = dict(leaf_size = leaf_size, n_neighbors = n_neighbors, p=p,metric = metric)

#I used GridSearch to look for the best parameters
from sklearn.model_selection import GridSearchCV

classifier = KNeighborsClassifier()

clf = GridSearchCV(classifier, hyperparameters,cv=10)
clf = clf.fit(X_train, y_train)

print('\n')

print(clf.best_params_)
print('\n')
clf = clf.fit(X_train, y_train)
# predicting using the model built

y_pred = clf.predict(X_test)
# comparing the actual and predicted

comparison_frame = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(comparison_frame)
comparison_frame.describe()
print('The accuracy of the model is ',metrics.accuracy_score(y_test, y_pred))
print('\n', 'Confusion matrix')
print(confusion_matrix(y_test, y_pred))
print('\n', 'Classification report')
print(classification_report(y_test, y_pred))



{'leaf_size': 35, 'metric': 'manhattan', 'n_neighbors': 3, 'p': 1}


     Actual  Predicted
14        0          1
159       0          0
763       1          1
741       0          1
483       1          0
..      ...        ...
620       0          0
821       1          0
65        1          0
884       0          0
52        1          1

[267 rows x 2 columns]
The accuracy of the model is  0.6891385767790262

 Confusion matrix
[[122  35]
 [ 48  62]]

 Classification report
              precision    recall  f1-score   support

           0       0.72      0.78      0.75       157
           1       0.64      0.56      0.60       110

    accuracy                           0.69       267
   macro avg       0.68      0.67      0.67       267
weighted avg       0.69      0.69      0.69       267



**Baseline Model - 60/40 split**

In [25]:
# split the data into labels and features

X = train[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare','Embarked']]
y = train['Survived']


# standardizing the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit_transform(X)


# splitting into train an test sets

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.4, random_state=0)

# setting the dicision tree classifier

# Modelling for training and predictions

clf = KNeighborsClassifier(n_neighbors=3)
clf = clf.fit(X_train, y_train)
# predicting using the model built

y_pred = clf.predict(X_test)

# comparing the actual and predicted

comparison_frame = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

comparison_frame.describe()

,Actual,Predicted
count,356.00,356.00
mean,0.39,0.36
std,0.49,0.48
min,0.00,0.00
25%,0.00,0.00
50%,0.00,0.00
75%,1.00,1.00
max,1.00,1.00


In [26]:
# Evaluating the Algorithm
print('The accuracy of the model is ',metrics.accuracy_score(y_test, y_pred))
print('\n', 'Confusion matrix')
print(confusion_matrix(y_test, y_pred))
print('\n', 'Classification report')
print(classification_report(y_test, y_pred))

The accuracy of the model is  0.6544943820224719

 Confusion matrix
[[161  55]
 [ 68  72]]

 Classification report
              precision    recall  f1-score   support

           0       0.70      0.75      0.72       216
           1       0.57      0.51      0.54       140

    accuracy                           0.65       356
   macro avg       0.63      0.63      0.63       356
weighted avg       0.65      0.65      0.65       356



**KNN with Grid search Model - 60/40 split**

In [27]:
#Defining the hyper parameters for the Knearest Neighbors Classifier
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p = [1,2]
metric = ['manhattan', 'euclidean', 'minkowski']
#Creating a dictionary with the hyperparameters
hyperparameters = dict(leaf_size = leaf_size, n_neighbors = n_neighbors, p=p,metric = metric)

#I used GridSearch to look for the best parameters
from sklearn.model_selection import GridSearchCV

classifier = KNeighborsClassifier()

clf = GridSearchCV(classifier, hyperparameters,cv=10)
clf = clf.fit(X_train, y_train)

print('\n')

print(clf.best_params_)
print('\n')

#Creating a better model based on the parameters given to us by the greadsearch
modelthree = KNeighborsClassifier(leaf_size = clf.best_params_['leaf_size'], n_neighbors = clf.best_params_['n_neighbors'], p = clf.best_params_['p'], metric = clf.best_params_['metric']).fit(X_train, y_train)
modelthree

# predicting using the model built

y_pred = modelone.predict(X_test)
# comparing the actual and predicted

comparison_frame = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print('\n')
print(comparison_frame)
print('\n')
print(comparison_frame.describe())



{'leaf_size': 15, 'metric': 'manhattan', 'n_neighbors': 9, 'p': 1}




     Actual  Predicted
14        0          1
159       0          0
763       1          1
741       0          1
483       1          0
..      ...        ...
839       1          1
820       1          1
101       0          0
704       0          0
739       0          0

[356 rows x 2 columns]


       Actual  Predicted
count  356.00     356.00
mean     0.39       0.37
std      0.49       0.48
min      0.00       0.00
25%      0.00       0.00
50%      0.00       0.00
75%      1.00       1.00
max      1.00       1.00


In [28]:
# Evaluating the Algorithm
print('The accuracy of the model is ',metrics.accuracy_score(y_test, y_pred))
print('\n', 'Confusion matrix')
print(confusion_matrix(y_test, y_pred))
print('\n', 'Classification report')
print(classification_report(y_test, y_pred))

The accuracy of the model is  0.7696629213483146

 Confusion matrix
[[180  36]
 [ 46  94]]

 Classification report
              precision    recall  f1-score   support

           0       0.80      0.83      0.81       216
           1       0.72      0.67      0.70       140

    accuracy                           0.77       356
   macro avg       0.76      0.75      0.76       356
weighted avg       0.77      0.77      0.77       356



In [29]:
# Lets compare the performance of our different models

# initialize list of lists
data = ['Baseline Model 80/20 split',  0.3539325842696629, 0.3539325842696629, 0.5949223346535772, '64.6%'], ['with Grid search Model - 80/20 split', 0.33146067415730335, 0.33146067415730335, 0.5757262145823338, '66.9%'], ['Baseline Model 70/30 split',  0.3333333333333333, 0.3333333333333333, 0.5773502691896257, '67.0%'], ['with Grid search Model - 70/30 split', 0.299625468164794, 0.299625468164794, 0.5473805515039734, '70%'], ['Baseline Model 60/40 split',  0.32303370786516855, 0.32303370786516855, 0.5683605439025201, '67.7%'], ['with Grid search Model - 60/40 split', 0.22752808988764045, 0.22752808988764045, 0.47699904600286197, '77%']
 
# Create the pandas DataFrame

df = pd.DataFrame(data, columns = ['Model_Name','Mean Absolute Error', 'Mean Squared Error','Root Mean Squared Error','Accuracy'])
 
# print dataframe.
print(df )

                             Model_Name  ...  Accuracy
0            Baseline Model 80/20 split  ...     64.6%
1  with Grid search Model - 80/20 split  ...     66.9%
2            Baseline Model 70/30 split  ...     67.0%
3  with Grid search Model - 70/30 split  ...       70%
4            Baseline Model 60/40 split  ...     67.7%
5  with Grid search Model - 60/40 split  ...       77%

[6 rows x 5 columns]


In [30]:
model_compare = pd.DataFrame(df )  #pd. DataFrame(np_array, columns=['Column1', 'Column2']) 
model_compare.set_index('Model_Name')

,Mean Absolute Error,Mean Squared Error,Root Mean Squared Error,Accuracy
Model_Name,,,,
Baseline Model 80/20 split,0.35,0.35,0.59,64.6%
with Grid search Model - 80/20 split,0.33,0.33,0.58,66.9%
Baseline Model 70/30 split,0.33,0.33,0.58,67.0%
with Grid search Model - 70/30 split,0.30,0.30,0.55,70%
Baseline Model 60/40 split,0.32,0.32,0.57,67.7%
with Grid search Model - 60/40 split,0.23,0.23,0.48,77%
